In [1]:
# External libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.style as style
import cv2
import time
import os
from utility.utils import *
import re
import h5py
import tables

# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

# CATALOG CREATION

In [13]:
DB = pd.read_csv("DATA/lunar_crater_database_robbins_2018.csv", sep=",")
DB['DIAM_CIRC_IMG'] *= 0.5

def craters_from_HLL(H,lat,lon, filter = 1):
    t = find_dteta(H)  # d/R, where d^2 is the footprint
    lat_bounds, lon_bounds = (np.array([lat - t/2, lat + t/2]), np.array([lon - t/2, lon + t/2]) )
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME="ROBBINS")
    df.rename(columns={'Diam':'r'}, inplace=True)
    df = df[ df.r > filter]        # km > filter
    df.r = df.r * 1/deg2km    #km2deg
    # Now it's all expressed in deg!
    return df

def HLL_catalog(df_mat):
    H = find_all_triplets(df_mat)
    Q = pd.DataFrame(H, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # Retrasform radius to km:
    Q.r1 *= deg2km
    Q.r2 *= deg2km
    Q.r3 *= deg2km
    Q.rename( columns={'x1':'lon1','x2':'lon2','x3':'lon3','y1':'lat1','y2':'lat2','y3':'lat3'}, inplace=True)
    return Q

In [15]:
lon, lat, H = -180, 0, 50

for i in range(2):
    df = craters_from_HLL(H,lat,lon)
    df_mat = np.array(df)
    cat = HLL_catalog(df_mat)
    if i == 0:
        CAT = cat 
    else:
        CAT = pd.concat([CAT,cat])

    lon +=1

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [19]:
CAT.to_hdf('test.h5', key='df', complevel=9, format='table')

In [ ]:
# # def image_gen(PATH):
# path = 'DATA/ephemeris sat/20 step size/IMG_20_Lat_0_Long_-127.683.jpg'

# # Actual Position:
# num =20
# l = -127.683
# H = 50
# X_c, Y_c = l, 0
# # x_f, y_f, z_f = spherical2cartesian(0, crater.Lat, crater.Lon)

# # Find reference in catalogs:
# dteta = find_dteta(H) # d/R, where d^2 is the footprint
# sp = dteta/2 # span
# lat_bounds, lon_bounds = np.array([Y_c-sp,Y_c+sp]), np.array([X_c-sp,X_c+sp])
# print('*'*35 +'BORDERS'+'*'*35)
# print(f'LAT: {lat_bounds}              LON:{lon_bounds}')
# print('*'*35 +'*******'+'*'*35)
# # 3 crater catalogs: HEAD | LROC | ROBBINS
# CATALOGS = ['ROBBINS', 'HEAD', 'LROC']
# for DATA in CATALOGS:
#     if DATA =='HEAD':
#         filepath = '/DATA/HeadCraters.csv'
#     elif DATA == 'LROC':
#         filepath = '/DATA/LROCCraters.csv'
#     elif DATA == 'ROBBINS':
#         filepath= 'DATA/lunar_crater_database_robbins_2018.csv'

#     DB = pd.read_csv(filepath, sep=',')
#     df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME=DATA)
#     if df is None:
#         print('No Craters found!')
#     else:     
#         print(f'Found {df.shape[0]} craters!\n\n', df)
    
#     ## DRAWINGS: 
#     ## FIG.1
#     img = cv2.imread(path)
#     plt.ion()
#     plt.figure('Tile and Mask -'+DATA, dpi=200)
#     plt.subplot(1,2,1)
#     plt.imshow(img)
#     plt.subplot(1,2,2)
#     I = draw_craters(df, lon_bounds, lat_bounds)
#     plt.imshow(I)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-A.png')
    
#     ## FIG.2
#     image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, img, u=None)
#     plt.ion()
#     plt.figure('Image with craters catalogued-'+DATA, dpi=200)
#     plt.imshow(image_with_craters)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-B.png')

# plt.close('all')